# Class 7 - Performance

Today we'll talk a bit about performance, in particular how to write Julia code with performance in mind, and how to measure performance.  You can find a lot of useful material in Julia's documentation: [FAQ](http://docs.julialang.org/en/release-0.4/manual/faq/), [performance tips](http://docs.julialang.org/en/release-0.4/manual/performance-tips/), [profiling](http://docs.julialang.org/en/release-0.4/manual/profile/).

We won't talk too much about performance relative to other languages (see Julia's figure [here](http://julialang.org/benchmarks/), and look around the internet for criticisms), and mostly concern ourselves with writing fast code within Julia.  However, many of these topics apply directly or indirectly to many languages used for scientific computing, so keep an eye out for questions or connections to languages you are familiar with.

A few things that we'll pay attention to today are speed and memory allocation.  A few general heuristics are:
* Preallocation is good (don't grow arrays dynamically if avoidable)
* Type annotations are good (tell the compiler which types you want to instantiate)
* Avoid changing the type of variables
* Write multiple function methods instead of multiple code paths in a function
* Use for-loops over vectorized notation (see [Victor's notebook](../../packages/devectorize.ipynb))

It is worth mentioning that if you don't want to worry about this sort of thing, that's OK.  One of the nice things about Julia is that you can use it at a high level without getting bogged down in this sort of analysis.  However, if you use certain functions a lot, or plan on having others use your functions a lot, or want your simulations to finish faster, it may be worth taking a second pass at your code to find optimizations.  Also, if you practice a bit, you will also be able to write code faster the first time around.


## Example: The Inner Product 

You've probably already used the `@time` macro, which is a very easy way to get an idea of speed and memory allocation.

In [ ]:
function bad_innerprod(x, y)
    @assert length(x) == length(y)
    ans = 0
    for i = 1:length(x)
        ans += x[i] * y[i]
    end
    return ans
end
;

In [ ]:
n = 1000
x = randn(n)
y = randn(n)
;

In [ ]:
@time bad_innerprod(x, y)

As you can see, the `@time` macro will tell you both time information and memory allocation information.  The function above is allocating several kilobytes of memory just to do some simple artihmetic.  This may not seem like a big deal, but imagine if this function was a very small component of a much larger program and was called thousands of times.  Here's a better example

In [ ]:
function better_innerprod0{T}(x::Array{T}, y::Array{T})
    @assert length(x) == length(y)
    ans = zero(T)
    for i = 1:length(x)
        ans += x[i] * y[i]
    end
    return ans
end
;

In [ ]:
@time better_innerprod0(x,y)

Essentially just by providing type information, we were able to keep the compiler from allocating unneccssary amounts of memory.

In [ ]:
function better_innerprod1{T}(x::Array{T}, y::Array{T})
    @assert length(x) == length(y)
    ans = zero(T)
    for i = 1:length(x)
        @inbounds ans += x[i] * y[i]
    end
    return ans
end
;

In [ ]:
@time better_innerprod1(x, y)

The `@inbounds` macro is saying that the program doesn't need to check that we may try to access a memory location that isn't part of the array.  The complier may be able to infer this in this particular example, but if you have more complicated loops, the macro may give you a noticeable speedup.

Now, we separate the inner loop from bounds checking.  If you have more complex logic, breaking your functions into smaller components can speed up evaluation.

In [ ]:
function fast_innerprod{T}(x::Array{T}, y::Array{T})
    ans::T = 0
    for i = 1:length(x)
        @inbounds ans += x[i] * y[i]
    end
    ans
end

function better_innerprod2{T}(x::Array{T}, y::Array{T})
    @assert length(x) == length(y)
    fast_innerprod(x, y)
end
;

In [ ]:
@time fast_innerprod(x, y)
@time better_innerprod2(x, y)

The `@simd` macro can be used in loops that can be vectorized.  This means no `break`s or `continue`s, and that the loop should not depend on previous loop evaluations.  See more [here](https://en.wikipedia.org/wiki/SIMD).

In [ ]:
function fast_innerprod2{T}(x::Array{T}, y::Array{T})
    ans::T = 0
    @simd for i = 1:length(x)
        @inbounds ans += x[i] * y[i]
    end
    ans
end

function better_innerprod3{T}(x::Array{T}, y::Array{T})
    @assert length(x) == length(y)
    fast_innerprod2(x, y)
end
;

In [ ]:
@time fast_innerprod2(x, y)
@time better_innerprod3(x, y)

You can also use the equivalent of the `-ffast-math` compiler optimization flag with `@fastmath`.  Note that this may change the accuracy of your results, or give you an answer that is entirely wrong if you aren't careful.

In [ ]:
function fast_innerprod3{T}(x::Array{T}, y::Array{T})
    ans::T = 0
    @fastmath @simd for i = 1:length(x)
        @inbounds ans += x[i] * y[i]
    end
    ans
end

function better_innerprod4{T}(x::Array{T}, y::Array{T})
    @assert length(x) == length(y)
    fast_innerprod3(x, y)
end
;

In [ ]:
@time fast_innerprod3(x, y)
@time better_innerprod4(x, y)

Here's how you would compute an inner product with Julia's built in dot: (note that this is calling BLAS).

In [ ]:
@time x' * y 
@time dot(x,y)


Now, let's compare the efficiency of each implementation

In [ ]:
function gflop_innerprod( n, reps )
    x = randn(n)
    y = randn(n)
    s = zero(Float64)
    time = @elapsed for j in 1:reps
        s+=bad_innerprod(x,y)
    end
    println("GFlop (bad_innerprod)      = ",2.0*n*reps/time*1E-9)
    time = @elapsed for j in 1:reps
        s+=better_innerprod0(x,y)
    end
    println("GFlop (better_innerprod0)  = ",2.0*n*reps/time*1E-9)
    time = @elapsed for j in 1:reps
        s+=better_innerprod1(x,y)
    end
    println("GFlop (better_innerprod1)  = ",2.0*n*reps/time*1E-9)
    time = @elapsed for j in 1:reps
        s+=better_innerprod2(x,y)
    end
    println("GFlop (better_innerprod2)  = ",2.0*n*reps/time*1E-9)
    time = @elapsed for j in 1:reps
        s+=better_innerprod3(x,y)
    end
    println("GFlop (better_innerprod3)  = ",2.0*n*reps/time*1E-9)
    time = @elapsed for j in 1:reps
        s+=better_innerprod4(x,y)
    end
    println("GFlop (better_innerprod4)  = ",2.0*n*reps/time*1E-9)  
    time = @elapsed for j in 1:reps
        s+=dot(x,y)
    end
    println("GFlop (dot)                = ",2.0*n*reps/time*1E-9)
        time = @elapsed for j in 1:reps
        s+=x' * y
    end
    println("GFlop (array inner prod)   = ",2.0*n*reps/time*1E-9)
end

In [ ]:
gflop_innerprod(2000, 1000)
println("")
gflop_innerprod(2000, 1000)

## Exercise 1

* Switch the $+$ and $*$ operations in the definition of inner product, and write a function that will compute this modified function for you.  Make one version that is relatively inneficient and one version that is as fast as you can make it.
* (if you have time) If you make the binary operations in the defintion of matrix-vector multiplication parameters of your function can you still get reasonable performance?

## Profiling Code

Julia has a built-in profiler which will allow you to see where your functions are spending most of their time.  If you have a script or function that is taking a long time to complete, this can help you identify where you should focus your optimization efforts.

In [ ]:
function test_fn()
    A = randn(1000, 1000)
    b  = randn(1000, 1)
    c = A * b
    maximum(c)
end

test_fn()

In [ ]:
@profile test_fn()

In [ ]:
Profile.print()

The `@profile` macro will run the function several times, randomly interrupting the call and looking at the stack.  The first number in each line is the number of times the function was found on the call stack.  The rest of the line gives you information on the function and where to find it in the code base.  The output is indented based on where in the stack the function was found.

If you want to increase the number of samples, you can put your function in a for-loop as follows:

In [ ]:
@profile for i = 1:100 test_fn() end

In [ ]:
Profile.print()

If you'd like to go beyond the built in profiler, there's a pacakge that will graphically interperet the results called [ProfileView](https://github.com/timholy/ProfileView.jl).  You can [track memory allocation](http://docs.julialang.org/en/release-0.4/manual/profile/#man-track-allocation) for each line of code by starting up Julia with `--track-allocation=<setting>`.

## Exercise 2

* Use Julia's profiler on the package of your choice.  What's taking the most time?  If you want a starting point, try PyCall.
* (if you have time) Use the profiler on your Julia set package.  Where would you focus your efforts if you wanted increased speed?

## More speed

### [Type Definitions](http://docs.julialang.org/en/release-0.4/manual/performance-tips/#type-declarations)

When you declare types, you should (whenever possible) make fields a concrete type, not even a specific abstract type.  If you want to allow for multiple types in the field, parameterize your type.  If there is any ambiguity in what the actual instantiated type will be, the compiler will not be able to allocate space correctly, and will generally miss out on optimizations.


In [ ]:
type AmbiguousType
    x
end

type StillAmbiguousType
    x::Real
end

type NonAmbiguousType
    x::Float64 
end

In [ ]:
n = 1000
for T in (AmbiguousType, StillAmbiguousType, NonAmbiguousType)
    @time a = Array(T, n)
    t1 = @elapsed for i=1:n
        a[i] = T(randn())
    end
    println("$t1 seconds to fill array")
    s = T(0)
    t2 = @elapsed for i=1:n
        s.x += (a[i]).x
    end
    println("$t2 seconds to sum array")
end

When you use arrays, you should pre-allocate if possible.  Specific type information is also valuable.

In [ ]:
n = 10000
@time a1 = Real[] # Abstract type
@time a2 = Float64[] # specific type
@time a3 = Array(Real, n) # pre-allocated abstract type
@time a4 = Array(Float64, n) # pre-allocated specific type
;

In [ ]:
t1 = @elapsed for i=1:n
    push!(a1,randn())
end
println("$t1 seconds to fill a1")
t2 = @elapsed for i=1:n
    push!(a2,randn())
end
println("$t2 seconds to fill a2")
t3 = @elapsed for i=1:n
    @inbounds a3[i] = randn()
end
println("$t3 seconds to fill a3")
t4 = @elapsed for i=1:n
    @inbounds a4[i] = randn()
end
println("$t4 seconds to fill a4")

### [Subnormal Numbers](http://docs.julialang.org/en/release-0.4/manual/performance-tips/#treat-subnormal-numbers-as-zeros)

You can treat sub-normal numbers as zero.  If a number is less than what can be represented using floating point, your computer may still represent it, and incur performance penalites (although this is required for IEEE standards, so be careful).  See [Denormal Numbers](https://en.wikipedia.org/wiki/Denormal_number) on Wikipedia for more info. The following example is from [Julia's documentation](http://docs.julialang.org/en/release-0.4/manual/performance-tips/#treat-subnormal-numbers-as-zeros), and models the heat equation.  

In [ ]:
function timestep{T}( b::Vector{T}, a::Vector{T}, Δt::T )
    @assert length(a)==length(b)
    n = length(b)
    b[1] = 1                            # Boundary condition
    for i=2:n-1
        b[i] = a[i] + (a[i-1] - T(2)*a[i] + a[i+1]) * Δt
    end
    b[n] = 0                            # Boundary condition
end

function heatflow{T}( a::Vector{T}, nstep::Integer )
    b = similar(a)
    for t=1:div(nstep,2)                # Assume nstep is even
        timestep(b,a,T(0.1))
        timestep(a,b,T(0.1))
    end
end

heatflow(zeros(Float32,10),2)           # Force compilation
for trial=1:6
    a = zeros(Float32,1000)
    set_zero_subnormals(iseven(trial))  # Odd trials use strict IEEE arithmetic
    @time heatflow(a,1000)
end

### [Access arrays in column-major order](http://docs.julialang.org/en/release-0.4/manual/performance-tips/#access-arrays-in-memory-order-along-columns)
If you need to loop over an array, keep in mind that it is stored in column-major format, so looping over indices in reverse order will allow you to use blocks of memory more efficiently.

In [ ]:
function sum_array1{T}(A::Array{T,3})
    s::T = 0
    @simd for k=1:size(A,3)
        @simd for j=1:size(A,2)
            @simd for i=1:size(A,1)
                @inbounds s += A[i,j,k]
            end
        end
    end
    return s
end

function sum_array2{T}(A::Array{T,3})
    s::T = 0
    @simd for i=1:size(A,1)
        @simd for j=1:size(A,2)
            @simd for k=1:size(A,3)
                @inbounds s += A[i,j,k]
            end
        end
    end
    return s
end
;

In [ ]:
n = 300
A = rand(Int64,n,n,n)
@time sum_array1(A)
@time sum_array2(A)
;

In [ ]:
A = [1 2; 3 4]
@show A
@show A[:]
;

### [Minor Tweaks](http://docs.julialang.org/en/release-0.4/manual/performance-tips/#tweaks)

Julia's performance documentation suggests the following optimizations for making very fast inner loops:

* Avoid unnecessary arrays. For example, instead of sum([x,y,z]) use x+y+z.
* Use `abs2(z)` instead of `abs(z)^2` for complex z. In general, try to rewrite code to use `abs2()` instead of `abs()` for complex arguments. (This would be useful for writing fast Julia Set codes!)
* Use `div(x,y)` for truncating division of integers instead of `trunc(x/y)`, `fld(x,y)` instead of `floor(x/y)`, and `cld(x,y)` instead of `ceil(x/y)`.

In [ ]:
n = 100000
@time z = Array(Complex{Float64}, n)
t = @elapsed for i = 1:n
    @inbounds z[i] = Complex{Float64}(randn(),randn())
end
println("$t sec to initialize")

In [ ]:
bnd = 0.5
s = zero(Int64)
t = @elapsed for i = 1:n
    @inbounds s += (abs(z[i]) > bnd ? one(Int64) : zero(Int64))
end
println("$t sec using abs()")
bnd2 = bnd*bnd
s = zero(Int64)
t = @elapsed for i = 1:n
    @inbounds s += (abs2(z[i]) > bnd2 ? one(Int64) : zero(Int64))
end
println("$t sec using abs2()")

## Exercise 3

* Write a standard matrix-vector multiplication function on two arrays.  Can you get close to Julia's default implementation's performance?
* How would you modify your routine to do a matrix transpose-vector multiplication routine? 
* Why do you think [BLAS's gemv](http://www.netlib.org/lapack/explore-html/dc/da8/dgemv_8f.html) takes the arguments that it does?

## Additional Performance Analysis

* [Lint](https://github.com/tonyhffong/Lint.jl) - analyze code for potential improvements
* [TypeCheck](https://github.com/astrieanna/TypeCheck.jl) - static analysis of types and function calls